In [3]:
import torch

from quiet_star.config import Config, ModelConfig
from quiet_star.torch.pretrained import PretrainedThoughtModel

In [9]:
config = Config(
    batch_size=1,
    epochs=2,
    lookahead_tokens=4,
    num_thoughts=2,
    seed=1,
    thought_length=10,
    model=ModelConfig(
        attn_type="torch",
        device=torch.device("cuda"),
        dropout_attn=0.0,
        dropout_embed=0.0,
        dtype="bfloat16",
        embed_dim=64 * 6,
        max_length=64,
        model_name="Qwen/Qwen1.5-0.5B-Chat",
        num_heads=6,
        num_layers=8,
    ),
)
model = PretrainedThoughtModel.load_from_checkpoint(
    "lightning_logs/version_19/checkpoints/epoch=1-step=3584.ckpt",
    config=config,
)
model.eval()

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


number of parameters: 468.19M


PretrainedThoughtModel(
  (model): Qwen2ForCausalLM(
    (model): Qwen2Model(
      (embed_tokens): Embedding(151936, 1024)
      (layers): ModuleList(
        (0-23): 24 x Qwen2DecoderLayer(
          (self_attn): Qwen2SdpaAttention(
            (q_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (k_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (v_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (o_proj): Linear(in_features=1024, out_features=1024, bias=False)
            (rotary_emb): Qwen2RotaryEmbedding()
          )
          (mlp): Qwen2MLP(
            (gate_proj): Linear(in_features=1024, out_features=2816, bias=False)
            (up_proj): Linear(in_features=1024, out_features=2816, bias=False)
            (down_proj): Linear(in_features=2816, out_features=1024, bias=False)
            (act_fn): SiLU()
          )
          (input_layernorm): Qwen2RMSNorm()
          (post_attention_layernorm): Qwen

In [45]:
text = """Bayes and his Theorem My earlier post on Bayesian probability seems to have generated quite a lot of readers, so this lunchtime I thought I'd add a little bit of background. The previous discussion started from """ # the result $P(B|AC) = K^{-1}P(B|C)P(A|BC) = K^{-1} P(AB|C)$ where $K=P("""

x = model.tokenizer(
    text,
    padding="do_not_pad",
    truncation=True,
    max_length=63,
    return_tensors="pt",
    return_attention_mask=False,
)["input_ids"][0].tolist()
x.append(model.start_thought_token_id)
x = torch.tensor([x], dtype=torch.int64, device=torch.device("cuda"))

xp = x[:, :-1]
logits = model(xp)
next_id = model.sample_next_tokens(logits, temp=0.1)[0, -1]
xp = torch.tensor([xp.tolist()[0] + [next_id]], dtype=torch.int64, device=torch.device("cuda"))

print(model.tokenizer.decode(xp[0]))

for _ in range(10):
    logits = model(x)
    next_id = model.sample_next_tokens(logits, temp=0.1)[0, -1]
    x = torch.tensor([x.tolist()[0] + [next_id]], dtype=torch.int64, device=torch.device("cuda"))

x = torch.tensor([x.tolist()[0] + [model.end_thought_token_id]], dtype=torch.int64, device=torch.device("cuda"))
logits = model(x)
next_id = model.sample_next_tokens(logits, temp=0.1)[0, -1]
x = torch.tensor([x.tolist()[0] + [next_id]], dtype=torch.int64, device=torch.device("cuda"))

print(model.tokenizer.decode(x[0]))


Bayes and his Theorem My earlier post on Bayesian probability seems to have generated quite a lot of readers, so this lunchtime I thought I'd add a little bit of background. The previous discussion started from 1
Bayes and his Theorem My earlier post on Bayesian probability seems to have generated quite a lot of readers, so this lunchtime I thought I'd add a little bit of background. The previous discussion started from <|startofthought|> Bayes, which is a probabilistic model that<|endofthought|>ens


In [31]:
y = model.sample_next_tokens(logits, temp=0.01)
next_token = model.tokenizer.decode(y[0, -1])

new_text = text + next_token

' Bay'